In [3]:
"""
This code processes geospatial data related to population, precipitation, and droughts. It loads population
 data from TIFF files, precipitation data from ERA5 NetCDF files, and World Bank country data.
 It rasterizes vector datasets, computes zonal statistics, and compiles these statistics over time.
 It generates annual drought data from SPI and SPEI indices, saving the results in NetCDF and Feather
 files for further analysis.

Input:
- ERA5_monthly_1970-2021_SPI-SPEI.nc
- Files in D:\Datasets\Gridded Population of the World
- world_bank_adm2.zip

Output:
- ERA5_yearly_1970-2021_SPI-SPEI.nc (intermediate files)
- ERA5_droughts_yearly.nc (final files)
- WB_country_grid.nc
- WB_country_IDs.feather

"""

"""
New packages used:

- hashlib: defines a programming interface for accessing various cryptographic hash algorithms.
- xrspatial: spatial analysis functions integrated with Xarray, useful for raster analysis and
 geographic data manipulation
"""

import os
import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
import hashlib

import dask
import xarray as xr
import xrspatial
from dask.diagnostics import ProgressBar
from geocube.api.core import make_geocube

import matplotlib.pyplot as plt
import seaborn as sns

PATH = "D:\World Bank\CLIENT v2"
DATA_RAW = rf"{PATH}\Data\Data_raw"
DATA_PROC = rf"{PATH}\Data\Data_proc"
DATA_OUT = rf"{PATH}\Data\Data_out"
GPW_PATH = rf"D:\Datasets\Gridded Population of the World"

c:\Users\ofici\AppData\Local\Programs\Python\Python311\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 12.0.1. Please consider upgrading.
  warnings.warn(


In [4]:
# floods = pd.read_csv(rf"{DATA_RAW}\Floods\GloFAS_floods.csv")

def load_population_data(bounds=None, generate=False):
    """
    This function processes population data by:
    1. Reading multiple TIFF files from the specified directory.
    2. Extracting the year from each filename and adding it as a coordinate to the dataset.
    3. Concatenating the datasets along the 'year' dimension into a single dataset.
    4. Filtering the dataset to include only data within the provided geographical bounds (if any).
    5. Cleaning the dataset by selecting the first band and removing the 'band' variable.
    6. Optionally saving the processed raster files if the 'generate' parameter is set to True.
    """

    print("Processing Population data...")
    
    # Select all files in GPW folder
    files = os.listdir(GPW_PATH)
    files = [f for f in files if f.endswith(".tif")]
    
    # Compile into a single dataset
    dss = []
    for f in tqdm(files):
        
        ds = xr.open_dataset(os.path.join(GPW_PATH, f), chunks={"x": 10000, "y": 10000})
        ds["band_data"] = ds["band_data"].astype(np.uint32)
        if bounds is not None:
            ds = ds.sel(
                x=slice(bounds[0], bounds[2]), y=slice(bounds[3], bounds[1])
            )
        if generate:
            with ProgressBar():
                ds.sel(band=1).drop_vars("band").band_data.rio.to_raster(rf"{DATA_PROC}\{f.replace('.tif','_proc.tif')}")
                print(f"Saved {f.replace('.tif','_proc.tif')}")
        
        ds["year"] = int(f.split("_")[5])
        ds = ds.set_coords('year')
        dss += [ds]
        
    population = xr.concat(dss, dim="year")    
    
    # Filter if bounds are provided
    if bounds is not None:
        population = population.sel(
            x=slice(bounds[0], bounds[2]), y=slice(bounds[3], bounds[1])
        )
        
    # Clean band dimension
    population = population.sel(band=1).drop_vars(["band"])
    
    print("Done!")
    return population

def load_precipitation_data():
    """
    This function processes precipitation data by
    1. Opening a NetCDF file containing ERA5 monthly precipitation data for the years 1970-2021, with SPI and SPEI
    indices.
    2. Chunking the data using 100x100 blocks for latitude and longitude dimensions.
    3. Renaming the dimensions from 'latitude' to 'y' and 'longitude' to 'x'.

    """
    era5 = xr.open_dataset(
        rf"{DATA_OUT}\ERA5_monthly_1970-2021_SPI-SPEI.nc",
        chunks={"latitude": 100, "longitude": 100},
    )
    era5 = era5.rename({"latitude": "y", "longitude": "x"})
    return

def load_WB_country_data(drop_adm2_na=False):
    """
    This function loads and processes World Bank country data by:
    1. Reading a compressed file containing country data.
    2. Merges the data by combining geometries that share the same ADMLAST_CODE, ADMLAST_NAME, ADM0_CODE,
    and ADM0_NAME into single geometries.
    5. Creates a unique ID for each combination of ADMLAST_CODE, ADMLAST_NAME, ADM0_CODE, and ADM0_NAME.
    """
    print("Loading World Bank country data...")
    WB_country = gpd.read_file(rf"{DATA_RAW}\world_bank_adm2.zip")
    
    # Assign nan when ADM2 is not available 
    WB_country.loc[WB_country.ADM2_NAME == "Administrative unit not available", "ADM2_CODE"] = (
        np.nan
    )
    
    # Create ADM_LAST variable: ADM2_NAME if available, else ADM1_NAME
    WB_country["ADMLAST_CODE"] = WB_country.ADM2_CODE
    WB_country["ADMLAST_NAME"] = WB_country.ADM2_NAME
    WB_country.loc[WB_country.ADM2_CODE.isnull(), "ADMLAST_CODE"] = WB_country.ADM1_CODE
    WB_country.loc[ WB_country.ADM2_CODE.isnull(), "ADMLAST_NAME"] = WB_country.ADM1_NAME

    # Dissolve by ADM_LAST and country code
    WB_country = WB_country.dissolve(by=["ADMLAST_CODE", "ADMLAST_NAME", "ADM0_CODE", "ADM0_NAME"]).reset_index()
    
    # Create ID
    WB_country["ID"] = WB_country.groupby(["ADMLAST_CODE", "ADMLAST_NAME", "ADM0_CODE", "ADM0_NAME"]).ngroup()
    assert WB_country.ID.nunique() == WB_country.shape[0], "ID is not unique!, there's some bug in the code..."
    print("Data loaded!")
    return WB_country


def rasterize_shape_like_dataset(shape, dataset):
    """
    This function rasterizes a vector dataset to match the dimensions of a reference raster dataset.
    """
    print("Rasterizing shape...")
    raster = make_geocube(
        vector_data=shape,
        like=dataset,
    )
    # For some reason, like option is not working, so I have to manually add x and y
    assert (raster["x"].shape == dataset["x"].shape)
    assert (raster["y"].shape == dataset["y"].shape)
    raster["x"] = dataset["x"]
    raster["y"] = dataset["y"]
    raster = raster.drop_vars(["spatial_ref"])
    raster = raster.chunk({"x": 100, "y": 100})
    print("Done!")
    return raster

def compute_zonal_stats(dataset, shape, value_var, groupby_var, gridded_groups=None, stats_funcs=["sum"], delayed=True):
    """
    This function computes zonal statistics for a raster dataset based on a vector dataset.
    1. Rasterizes the vector dataset to match the raster dataset dimensions if no pre-rasterized groups are provided.
    2. Sets up and computes zonal statistics using the specified statistical functions.
    3. Merges the zonal statistics with the original vector dataset based on the groupby variable.
    4. Returns the combined DataFrame with zonal statistics.
    """
    # Rasterize shape
    if gridded_groups is None:
        gridded_groups = rasterize_shape_like_dataset(shape[[groupby_var, "geometry"]], dataset)

    # Compute zonal stats  
    assert gridded_groups.chunks is not None, "Please, chunk the dataset before computing zonal stats! (e.g. dataset.chunk({'x': 100, 'y': 100})). Otherwise, you will get a MemoryError."
    assert dataset.chunks is not None, "Please, chunk the dataset before computing zonal stats! (e.g. dataset.chunk({'x': 100, 'y': 100})). Otherwise, you will get a MemoryError."

    print("Setting up zonal stats...")
    pop_by_adm = xrspatial.zonal.stats(gridded_groups[groupby_var], dataset[value_var], stats_funcs=stats_funcs)
    print("Done! Computing zonal stats...")    
    if delayed:
        return pop_by_adm
    
    with ProgressBar():
        pop_by_adm = pop_by_adm.compute()
    
    # Format zonal_stats dataframe
    pop_by_adm = pop_by_adm.rename(columns={
        "sum": value_var,
        "mean": f"{value_var}_mean",
        "zone": groupby_var,
    })
    
    result = (
        shape[[groupby_var, "geometry"]]
        .merge(pop_by_adm, on=groupby_var)
    )
    return result 

def compute_zonal_stats_over_time(dataset, shape, value_var, groupby_var, population_data=None, gridded_groups=None, stats_funcs=["mean"], delayed=True):
    """
    This function computes zonal statistics for a raster dataset over time based on a vector dataset.
    1. Rasterizes the vector dataset to match the raster dataset dimensions if no pre-rasterized groups are provided.
    2. Sets up and accumulates zonal statistics tasks for each year in the raster dataset.
    3. If population data is provided, weights the raster values by population.
    4. Returns the combined DataFrame with zonal statistics.
    """
    import warnings
    warnings.filterwarnings("ignore", category=RuntimeWarning)
    
    # Rasterize shape
    if gridded_groups is None:
        gridded_groups = rasterize_shape_like_dataset(shape[[groupby_var, "geometry"]], dataset)

    # Compute zonal stats  
    assert gridded_groups.chunks is not None, "Please, chunk the dataset before computing zonal stats! (e.g. dataset.chunk({'x': 100, 'y': 100})). Otherwise, you will get a MemoryError."
    assert dataset.chunks is not None, "Please, chunk the dataset before computing zonal stats! (e.g. dataset.chunk({'x': 100, 'y': 100})). Otherwise, you will get a MemoryError."
    assert "year" in dataset.dims, "Please, add a 'year' dimension to the dataset before computing zonal stats! (e.g. dataset = dataset.assign_coords(year=dataset.time.dt.year))."
 
    print("Setting up zonal stats...")

    tasks = []
    for year in tqdm(dataset["year"].values):
        dataset_year = dataset.sel(year=year).drop_vars("year")
        if population_data is not None:
            dataset_year = dataset_year * population_data.sel(year=year, method="nearest").drop_vars("year")
        else:
            dataset_year[value_var] = dataset_year[value_var].astype("float32")

        tasks += [xrspatial.zonal.stats(zones=gridded_groups[groupby_var], values=dataset_year[value_var], stats_funcs=stats_funcs)]
    if delayed:
        return tasks

    print("Done! Computing zonal stats...")        
    with ProgressBar():
        result = dask.compute(*tasks)
        
    return result 

def compile_zonal_stats_over_time(tasks_results, shape, groupby_var, value_var):
    """
    This function compiles the results of zonal statistics calculated over time into a single DataFrame.
    1. Creates a dictionary where the keys are years (1970-2020) and the values are the zonal statistics results
    with the 'zone' column set as the index.
    2. Concatenates the dictionary of DataFrames into a single DataFrame with a MultiIndex that includes the years.
    3. Resets the index, converting the levels of the index (years and zones) into columns.
    4. Merges the compiled zonal statistics DataFrame with the original vector dataset based on the 'groupby_var'.
    5. Returns the combined DataFrame with zonal statistics.
    """
    # Compile results into a single df
    out_dict = {year: data.set_index("zone") for year, data in zip(range(1970,2021), tasks_results)}
    df = pd.concat(out_dict)
    df = df.reset_index()
    df = df.rename(columns={"level_0":"year"})
    
    # Format zonal_stats dataframe
    df = df.rename(columns={
        "sum": value_var,
        "mean": value_var,
        "zone": groupby_var,
    })
    
    result = (
        shape[[groupby_var, "geometry"]]
        .merge(df, on=groupby_var)
    )
    return result

def process_era5_data():
   
    # Load ERA5 data
    
    # Create droughts dummies
    
    # Annualize series
    
    return

# Procesa WB country Data (administrative boundaries) y GPW (population data)

In [ ]:
WB_country = load_WB_country_data()
population = load_population_data(bounds=WB_country.total_bounds)

# # Rasterize WB_country
# WB_country_grid = rasterize_shape_like_dataset(
#     WB_country[["ID", "geometry"]], 
#     population
# )

# WB_country_path = rf"E:\client_v2_data\WB_country_grid.nc"
# print("Saving WB_country_grid...")
# with ProgressBar():
#     WB_country_grid.to_netcdf(WB_country_path)
        
# WB_country[["ID", "OBJECTID", "ADM2_CODE", "ADM2_NAME", "ADM1_CODE", "ADM1_NAME", "ADM0_CODE", "ADM0_NAME", "ADMLAST_CODE", "ADMLAST_NAME", "geometry"]].to_feather(rf"E:\client_v2_data\WB_country_IDs.feather")

# GENERA Base de shocks, sin interpolar

In [5]:
import utils
droughts_path = rf"{DATA_OUT}\ERA5_droughts_yearly.nc"

In [ ]:
# if not os.path.exists(droughts_path):
print("Preparing droughts dataset...")
# Genera base de sequías
era5 = xr.open_dataset(rf"{DATA_OUT}\ERA5_monthly_1970-2021_SPI-SPEI.nc", chunks={'latitude': 1000, 'longitude': 1000})
# Corrije la dimensión x, que va de 0 a 360
era5 = era5.rename({'latitude': 'y', 'longitude': 'x'})
era5 = utils.coordinates_from_0_360_to_180_180(era5) # FIXME: no se si esto está andando bien, pero creo que si. VERIFICAR

# Calcula las sequías anuales
spi_yearly = era5.groupby("time.year").min()
with ProgressBar():
    spi_yearly.to_netcdf(rf"E:\client_v2_data\ERA5_yearly_1970-2021_SPI-SPEI.nc")

In [6]:
spi_yearly = xr.open_dataset(rf"{DATA_PROC}\ERA5_yearly_1970-2021_SPI-SPEI.nc", chunks={"x": 900, "y": 1800})

spi_spei_vars = [var for var in spi_yearly.data_vars if "-" in var]
for var in spi_spei_vars:
    for threshold_str in ["1_0", "1_5", "2_0", "2_5"]:
        threshold = float(threshold_str.replace("_", "."))
        threshold_str = threshold_str.replace("_", "")
        spi_yearly[f"drought_{var}_{threshold_str}sd"] = (spi_yearly[var] < -threshold).astype("bool")

spi_yearly = spi_yearly[[var for var in spi_yearly.data_vars if "drought" in var]]
spi_yearly = spi_yearly.rename({
    var: var.replace("drought_", "").replace("-", "") for var in spi_yearly.data_vars
})
# with ProgressBar():
#     spi_yearly.to_netcdf(droughts_path)

c:\Users\ofici\AppData\Local\Programs\Python\Python311\Lib\site-packages\xarray\backends\plugins.py:80: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


In [8]:
spi_yearly.x.sel(x=slice(-1, 1))

<xarray.DataArray 'x' (x: 20)> Size: 160B
array([-0.90025 , -0.800222, -0.700194, -0.600167, -0.500139, -0.400111,
       -0.300083, -0.200056, -0.100028,  0.      ,  0.      ,  0.100028,
        0.200056,  0.300083,  0.400111,  0.500139,  0.600167,  0.700194,
        0.800222,  0.90025 ])
Coordinates:
  * x        (x) float64 160B -0.9003 -0.8002 -0.7002 ... 0.7002 0.8002 0.9003